In [1]:
import json
import csv
import time
import math
import sys
from datetime import datetime, timezone

import requests

import pandas as pd

def get_data(pair, interval, start_date, end_date, exchange="binance"):
    """
    pair: for example ETHBTC
    exchange: for example: binance
    interval: granularity, for example: 1d, 1h
    number: number of candles required
    """   
    URL = "https://cryptocandledata.com/api/candles"
    MAX_COUNT = 4

    data = []
    done = False

    while not done:
        
        payload = {
            "exchange": "binance",
            "tradingPair": pair,
            "interval": interval,
            "startDateTime": start_date,
            "endDateTime": end_date
        }

        r = requests.get(URL, params=payload)
        
        data += r.json()["candles"]
        
        start_date = int(data[-1]["timestamp"]) // 1000 
        
        if start_date >= end_date - (1 * 3600 * 24):
            done = True
        
        time.sleep(10)
        
           
    df = pd.DataFrame(data)
    df = df.drop_duplicates(subset='timestamp', keep="first")

#     plt.plot(df["timestamp"].values)
#     plt.show()

    filename = "data/{}-{}.csv".format(pair, interval)
    
#     try:
    df.to_csv(filename, index=False)
#     except Exception as e:
#         print(e)
        

## Download a single file (pair + granularity)

In [5]:
PAIR = "BTCUSDT"
INTERVAL = "1d"
START_DATE = int(datetime(year=2020, month=1, day=1, hour=0, minute=0, second=0, tzinfo=timezone.utc).timestamp())
END_DATE = int(datetime(year=2021, month=5, day=1, hour=0, minute=0, second=0, tzinfo=timezone.utc).timestamp())



get_data(PAIR, INTERVAL, START_DATE, END_DATE)

## Download multiple files

In [ ]:
pairs = [
    "BTCUSDT",
    "ETHUSDT",
    "BNBUSDT",
    "XRPUSDT",
    "DOGEUSDT",
]

intervals = [
    "1h", "1d"
#     '15m', '30m', "1h",
]

START_DATE = int(datetime(year=2020, month=1, day=1, hour=0, minute=0, second=0, tzinfo=timezone.utc).timestamp())
END_DATE = int(datetime(year=2021, month=10, day=1, hour=0, minute=0, second=0, tzinfo=timezone.utc).timestamp())


for pair in pairs:
    for interval in intervals:
        print(pair, interval)
        
        done = False
        while not done:
            try:
                get_data(pair, interval, START_DATE, END_DATE)
                done = True
            except Exception as e:
                print(e)
                print("Fail, trying again")
            time.sleep(60)    
        time.sleep(2)
            
print("Done")

BTCUSDT 1h


## Validate files

In [21]:
import glob
import os

import numpy as np
import pandas as pd

PATH = "data"

paths = glob.glob(os.path.join(".", PATH, "*.csv"))

# paths = [path for path in paths if "1m" in path]


for name in paths:

    
    data = pd.read_csv(name, header=0)

    # implement data validity check

{86400000}